# Binaryzacja

### Zadanie domowe - binaryzacja adaptacyjna w oknach z interpolacją.

Pokazana w ramach podstawowej części ćwiczenia binaryzacja adaptacyjna działa dobrze, ale jest dość złożona obliczeniowo (choć oczywiście należy mieć świadomość, że implementację można zoptymalizować i wyeliminować pewne powtarzające się obliczenia).
Zbliżone rozwiązanie można również realizować w nieco innym wariancie - w oknach.
Ogólna idea jest następująca: wejściowy obraz dzielimy na nienachodzące (rozłączne) okna - wygodnie jest założyć, że są one kwadratowe i o rozmiarze będącym potęgą liczby 2.
W każdym z okien obliczamy próg - niech to będzie średnia i stosujemy do binaryzacji lokalnej.
Jak nietrudno się domyślić efekt nie będzie dobry, gdyż na granicach okien wystąpią artefakty.
Aby je wyeliminować należy zastosować interpolację, co zostanie szczegółowo opisane poniżej.
Warto zaznaczyć, że podobny mechanizm interpolacji stosowany jest w poznanym wcześniej algorytmie CLAHE.
Zainteresowane osoby odsyłam do artykułu na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization) oraz do artykułu o metodzie CLAHE - Zuiderveld, Karel. “Contrast Limited Adaptive Histograph Equalization.” Graphic Gems IV. San Diego: Academic Press Professional, 1994. 474–485.



Na początek zaimplementujemy wariant metody bez interpolacji:
1. Wczytaj obraz _rice.png_.
2. W dwóch pętlach `for`, dla okien o ustalonym wymiarze $W$ (potęga 2), oblicz średnią:
- pętle powinny mieć krok $W$,
- wynik (tj. średnie) należy zapisać w pomocniczej tablicy,
- przydatny operator to `//` - dzielenie całkowitoliczbowe (*floor division*).

3. W kolejnych dwóch pętlach `for` (tym razem o kroku 1) przeprowadź binaryzację z wyznaczonymi progami.
   Tu oczywiście należy się sprytnie odwołać do wyników z tablicy pomocniczej.
   Wyświetl wyniki - czy jest on poprawny?
   Podpowiedź - błędy lepiej widać dla mniejszego rozmiaru okna (np. 16 x 16).

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

if not os.path.exists("rice.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/rice.png --no-check-certificate
        
rice = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE)

W = 2**4
(X,Y) = rice.shape
bin = np.zeros((X,Y))

pom = np.zeros((int(X/W),int(Y/W)))


for i in range(0, X-W, W):
    for j in range(0, Y-W, W):

        seg = rice[i:i+W+1,j:j+W+1]
        pom[i//W][j//W] = np.mean(seg)

print(pom.shape)
print(X,Y)

for i in range(X):
  for j in range(Y):
    bin[i][j]=rice[i][j] > pom[i//W][j//W]
    bin[i][j] = bin[i][j].astype(np.int)



plt.imshow(rice)
plt.gray()
plt.axis('off')
plt.title("obraz orginalny")
plt.show()
plt.imshow(bin)
plt.axis('off')
plt.title("obraz po przekształceniach")
plt.show()


4. Rozwiązaniem problemu artefaktów na obrazie jest zastosowanie interpolacji.
   Próg binaryzacji dla danego okna wyliczany jest na podstawie progów z sąsiednich okien.
   ![Ilustracja koncepcji interpolacji](https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/clahe_tile_interpolation.png)

   Koncepcja została przedstawiona na powyższym rysunku.
   Możliwe są 3 przypadki:
   - piksel leży w rogach obrazu (kolor czerwony) - wtedy za próg przyjmuje się wartość średniej obliczonej dla danego okna,
   - piksel leży na krawędzi obrazu (kolor zielony) - wtedy za próg przyjmuje się wartość obliczoną na podstawie średnich z dwóch sąsiednich okien,
   - piksel leży w środku (kolor fioletowy) - wtedy próg jest obliczany na podstawie 4 sąsiednich okien.

   Uwaga. Proszę zwrócić uwagę, że sprawa jest dość złożona.
   Obraz dzielimy na okna (dla nich liczymy średnią) i następnie każde z okien "wirtualnie" na cztery sub-okna (linie przerywane).
   To ułatwia znalezienie środków okien (czarne kwadraty), które są wykorzystywane w interpolacji.

5. Implementujemy interpolację.
   Potrzebujemy do tego znać progi (jeden, dwa lub cztery), ale dla przejrzystości obliczeń lepiej zawsze przyjąć cztery oraz odległości od rozważnego piksela do środka sąsiednich okien (też w ogólnym przypadku 4):
   - całość sprowadza się do określania pozycji piksela,
   - na początek rozważmy przypadek czterech narożników (kolor czerwony na rysunku) - trzeba napisać `if`, który je wyznaczy,
   - warto sprawdzić, czy nie popełniliśmy błędu i np. tymczasowo do obrazu wynikowego w tym miejscu przypisać wartość 255. Efekt powinien być taki, że widoczne będą tylko narożniki.
   - drugi przypadek do brzegi (kolor zielony) - postępujemy podobnie jak przy narożnikach, przy czym osobno wydzielamy brzegi pionowe i poziome. Tu też warto sobie obrazek "pokolorować".
   - na koniec wyznaczamy piksele w środku.
   - analizując poprawność proszę zwrócić uwagę na to, żeby nie było przerw pomiędzy obszarami.
   - mając podział możemy dla każdego z obszarów wyliczyć cztery progi ($t11, t12, t21, t22$):
        - dla narożników wartość ta będzie identyczna i wynosi po prostu `t11 =t[jT][iT]`, gdzie `iT=i//W` oraz `jT=j//W`.
          Uwaga. Proszę używać indeksów tymczasowych $jT,iT$, gdyż będą potrzebne w dalszych obliczeniach.
        - dla brzegów pionowych występują dwie wartości: okno bieżące i sąsiednie.
          Wyznaczenie współrzędnej poziomej jest proste (jak dla narożników).
          Nad współrzędną pionową trzeba się chwilę zastanowić - aby nie rozważać wielu przypadków można od bieżącej współrzędnej odjąć połowę rozmiaru okna i dopiero później wykonać dzielenie przez rozmiar okna.
          W ten sposób otrzymujemy indeks okna o mniejszej współrzędnej.
          Indeks drugiego uzyskamy dodając 1.
          Proszę się zastanowić dlaczego to działa - najlepiej to sobie rozrysować.
        - dla brzegów poziomych należy postąpić analogicznie,
        - obliczenia dla obszaru wewnątrz powinny być już oczywiste.
   - kolejny krok to wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami.
     Przykładowo dla osi X wygląda to następująco: `dX1 = i - W/2 - iT*W` oraz `dX2 = (iT+1)*W - i-W/2`.
     Dla osi Y analogicznie.
     Ponownie proszę się zastanowić dlaczego to jest poprawne - najlepiej to sobie narysować.
   - ostatni krok to interpolacja dwuliniowa.
     Wykonamy ją w trzech krokach:
     - interpolacja w osi X dla dwóch górnych okien - sprowadza się ona do średniej ważonej pomiędzy wartościami $t11$ i $t12$, przy czym wagi to odpowiednio $dX2/W$ i $dX1/W$.
       Ponownie na podstawie rysunku proszę to przemyśleć.
     - interpolacja w osi X dla dolnych okien jest analogiczna,
     - interpolacja w osi Y również jest analogiczna, z tym, że wejściem są dwa wyniki interpolacji w poziomie.

6. "Kropka nad i" to oczywiście binaryzacja z wyznaczonym poprzez interpolację progiem - proszę dobrać rozmiar okna.
7. Na koniec proszę porównać na wspólnym rysunku poznane metody binaryzacji:
- Otsu,
- lokalna na podstawie średniej,
- lokalna Sauvoli,
- lokalna w oknach bez interpolacji,
- lokalna w oknach z interpolacją.

Proszę pod porównaniem, w osobnej sekcji *markdown*, krótko skomentować uzyskane wyniki.

In [ ]:

W = 2**4
(X,Y) = rice.shape
pom2 = np.zeros((X,Y))

bin2 = np.zeros((X,Y))

for i in range(X):
  for j in range(Y):

    iT=i//W
    jT=j//W
    t11 = 0
    t12 = 0
    t13 = 0
    t14 = 0

    if (i==0 or i == X-1) and (j == 0 or j == Y-1):
      t11 = pom[jT, iT]

      pom2[j][i] = t11


    elif (j == 0 or j == Y-1) and i!= 0 and i!= X-1:

      try:
        t12 = (pom[jT,int((i-W/2)//W)]+pom[jT,iT + 1])/2
      except IndexError:
        t12 = (pom[jT,int((i-W/2)//W)]+pom[jT,iT])/2  

      pom2[j][i] = t12


    elif (i == 0 or i == X-1) and j!= 0 and j!= Y-1:

      try:
        t12 = (pom[int((j-W/2)//W), iT]+pom[jT + 1, iT])/2
      except IndexError:
        t12 = (pom[int((j-W/2)//W), iT]+pom[jT, iT])/2

      pom2[j][i] = t13


    else:
      if j//W== 0:
        ff = pom[jT, iT]
      else:
        ff = pom[jT -1, iT]

      if i//W == 0:
        gg = pom[jT, iT]
      else:
        gg = pom[jT, iT-1]

      # if j//W == 255//W:
      #   first = pom[jT, iT]
      # else:
      #   first = pom[jT + 1, iT]

      # if i//W == 255//W:
      #   second = pom[jT, iT]
      # else:
      #   second = pom[jT, iT+1]

      try:
        first = pom[jT+1,iT]
      except IndexError:
        first = pom[jT,iT]
      try:
        second = pom[jT,iT+1]
      except IndexError:
        second = pom[jT,iT]





      t14 = (first + second + ff + gg)/4


      pom2[j][i] = t14

    # else:
    #   r11 = pom2[jT, (i - int(W/2)) // W]
    #   r21 = pom2[(j - int(W/2)) // W,iT]
    #   r12 =  pom2[jT,1 + (i - int(W/2)) // W]
    #   r22 =  pom2[1 + (j -int(W/2)) // W, iT]
      
    #   t14 = (r11 + r12 + r21 + r22)/4
    #   pom2[j][i] = t14

    # dX1 = i - W/2 - iT*W
    # dX2 = (iT+1)*W - i-W/2
    # dY1 = j - W/2 - jT*W
    # dY2 = (jT+1)*W - j-W/2




#NIEUDANA PRÓBA NR NAWET NIE WIEM JUŻ ILE
    # for i in range(0, X - int(W/2), int(W/2)):
    #   for j in range(0,Y - int(W/2), int(W/2)):

    #     if  (i//(W/2) == 0 or i//(W/2) == (X-1)//(W/2)) and (j//(W/2) == 0 or j//(W/2) == (Y-1)//(W/2)):
    #       # im1[j:j+int(W//2), i:i+int(W//2)] = 255
    #       t11 = pom[jT][iT]

    #     elif  (i//(W/2) == 0 or  i//(W/2) == (X-1)//(W/2))  and (j//(W/2) != 0 or j//(W/2) != (Y-1)//(W/2)):
    #       t12 = (pom[jT][int((i-W/2)//W)]+pom[jT][iT+1])/2

    #     elif  (j//(W/2) == 0 or  j//(W/2) == (X-1)//(W/2))  and (i//(W/2) != 0 or i//(W/2) != (Y-1)//(W/2)):
          
    #       t13 = (pom[int((j-W/2)//W)][iT]+pom[int(j+1)][iT])/2

#        else:
          # t14 = (pom[(int((j+1)//W))][iT]+pom[int((j-1))][iT]+pom[jT][int((i-1)//W)] + pom[jT][int((i+1)//W)])/4

#     # dX1 = i - W/2 - iT*W
#     # dX2 = (iT+1)*W - i-W/2

#     # dY1 = j - W/2 - jT*W
#     # dY2 = (jT+1)*W - j-W/2

#     # intX1 = (dX1/W*t12 + dX2/W*t11)/(dX1/W+dX2/W)
#     # intX2 = (dX1/W*t12 + dX2/W*t11)/(dX1/W+dX2/W)




#KOLEJNA PRÓBA
# for i in range(X):
#   for j in range(Y):

#     iT=i//W
#     jT=j//W


#     if (i==0 or i == X-1) and (j == 0 or j == Y-1):
#       t11 = pom[jT][iT]
#       t21 = pom[jT][iT]
#       t12 = pom[jT][iT]
#       t22 = pom[jT][iT]
#       pom2[j][i] = t11


#     elif (j == 0 or j == Y-1) and i!= 0 and i!= X-1:
#       t11 = pom[int((j-W/2)//W), iT]
#       t21 = pom[int((j-W/2)//W), iT]

#       try:
#         t12 = pom[int((j-W/2)//W) + 1, iT]
#         t22 = pom[int((j-W/2)//W) + 1, iT]
#       except IndexError:
#         t12 = pom[jT, iT]
#         t22 = pom[jT, iT]


#     elif (i == 0 or i == X-1) and j!= 0 and j!= Y-1:

#       t12 = pom[jT,int((i-W/2)//W)]
#       t22 = pom[jT,int((i-W/2)//W)]

#       try:
#         t11 = pom[jT,int((i-W/2)//W) + 1]
#         t21 = pom[jT,int((i-W/2)//W) + 1]
#       except IndexError:
#         t11 = pom[jT,iT] 
#         t21 = pom[jT,iT]

#     else:
#       try:
#        t12 = pom[int((j-W/2)//W)+1,iT]
#        t22 = pom[int((j-W/2)//W)-1, iT]
#       except IndexError:
#         t12 = pom[jT,iT]
#         t22 = pom[int((j-W/2)//W)-1, iT]
#       try:
#         t11 = pom[jT,int((i-W/2)//W)+1]
#         t21 = pom[jT, int((i-W/2)//W)-1]
#       except IndexError:
#         t11 = pom[jT,iT]
#         t21 = pom[jT, int((i-W/2)//W)-1]

      
#     dX1 = j - W/2 - jT*W
#     dX2 = (jT+1)*W - j+W/2
#     dY1 = i + W/2 - iT*W
#     dY2 = (iT+1)*W - i + W/2


#     pom2[i][j] = (t11*(dX2 / W) + t12*(dX1 / W) + t21 * (dY2 / W) + t22 * (dY1 / W)) / ((dX2 / W) + (dX1 / W) + (dY2 / W) + (dY1 / W))





for i in range(X):
  for j in range(Y):
    bin2[i][j] = rice[i][j] > pom2[i][j]
    bin2[i][j] = bin2[i][j].astype(np.int)



plt.imshow(rice)
plt.gray()
plt.axis('off')
plt.title("obraz orginalny")
plt.show()
plt.imshow(bin2)
plt.axis('off')
plt.title("obraz po przekształceniach")




In [ ]:
#POZNANE BINARYZACJE
def show (I, tit):
  plt.imshow(I)
  plt.axis('off')
  plt.title(tit)
  plt.show()

def binary(I, k = 256/2/2):

  bin = np.where(I < k, 0, 1)
  plt.imshow(bin,'gray')
  plt.axis('off')
  plt.title('obraz binarny')
  plt.show()

k_ots,bin_ots = cv2.threshold(rice,0,255, cv2.THRESH_OTSU)


def lokalna_srednia(rice, W):
  bin = np.zeros(rice.shape)
  (X,Y) = rice.shape
  for j in range(int(W/2), int(Y-W/2)):
    for i in range(int(W/2), int(X-W/2)):
      seg = rice[i-int(W/2):i+int(W/2),j-int(W/2):j+int(W/2)]
      m = np.mean(seg)
      bin[i][j] = rice[i][j] > m
      bin[i][j] = bin[i][j].astype(np.int)
      
  plt.gray()
  show(bin, "lokalna na podstawie średniej")

def Sauvol(rice):
  (X,Y) = rice.shape
  W = 16
  k = 0.15
  R = 128

  bin = np.zeros(rice.shape)

  for j in range(int(W/2), int(Y-W/2)):
      for i in range(int(W/2), int(X-W/2)):

        seg = rice[i-int(W/2):i+int(W/2),j-int(W/2):j+int(W/2)]
        T = np.mean(seg) * (1 - k*(np.std(seg)/R -1))
        bin[i][j] = rice[i][j] > T
        bin[i][j] = bin[i][j].astype(np.int)

        
  show(bin, "metoda Sauvol'i: W = 16")


In [ ]:

plt.imshow(rice)
plt.gray()
plt.axis('off')
plt.title("obraz orginalny")
plt.show()

plt.imshow(bin2)
plt.axis('off')
plt.title("lokalna w oknach z interpolacją")
plt.show()

plt.imshow(bin)
plt.axis('off')
plt.title('lokalna w oknach bez interpolacji')
plt.show()

Sauvol(rice)

lokalna_srednia(rice, 16)

show(bin_ots, 'Binaryzacja Otsu')
plt.show()

Binaryzacja za pomocą Otsu najlepiej pozbywa się "szumów świetlnych", i gdyby pozbywała się tylko ich, to byłaby najlepszym sposobem binaryzacji - niestety pozbywa się ona też elementów po prostu jasnych nie należących do tła - ziarenka ryżu w dolnych partiach są bardzo pomniejszone. 
Binaryzacja lokalna na podstawie średniej zachowuje bardzo dobrze kształt ziarenek ryżu, ale zaś tło jest bardzo zaszumione.
Binaryzacja metodą Sauvoliego jest ulepszenem binaryzacji lokalnej na podstawie średniej - kształt elementów jasnych jest zachowany, jak i tło pozostaje względnie jednolite. Według mnie właśnie działa najlepiej dla powyższego obrazka - jedynie następuje zmniejszenie obrazu ze wszystkich stron o szerokość ramki, w której to liczyliśmy potrzebne średnie. (podobnie jak w poprzedniej metodzie)
Binaryzacja w oknach bez interpolacji nie zmiejsza wielkości obrazu i dobrze zachowuje kształt elementów jasnych, jednak pojawia się zaszumienie w co poniektórych oknach.
Binaryzacja w oknach z interpolacją posiada zalety binaryzacji w oknach bez ineterpolacji oraz minimalizuje właśnie to zaszumienie okien - niestety tylko coś w mojej implementacji nie wyszło i prawa wraz z dolną krawędzią stają się nieczytelne.
